# Scripts to export jupyter notebook cells and run notebooks:


Each cell starting with "#export" in the abc_def.ipynb notebook
is written into the file ./exp/nb_abc.py by running  
``` bash
>>> python notebook2script.py abc_def.ipynb  
``` 

To run a notebook:

``` bash
>>> python run_notebook.py abc_def.ipynb
```


## Script notebook2script.py:



```python
import json, fire, re
from pathlib import Path

def is_export(cell):
    if cell['cell_type'] != 'code':
        return False
    src = cell['source']
    if len(src) == 0 or len(src[0])<7 :
        return False
    return re.match(r'^\s*#\s*export\s*$', src[0], re.IGNORECASE) is not None

def notebook2script(fname):
    fname = Path(fname)
    fname_out = f'nb_{fname.stem.split("_")[0]}.py'
    main_dic = json.load(open(fname, 'r'))
    code_cells = [c for c in main_dic['cells'] if is_export(c)]
    module = f'''
###############################################################
#### THIS FILE IS AUTOGENERATED! DO NOT EDIT! #################
###############################################################
# file to edit: dev_nb/{fname.name}    

'''
    for cell in code_cells: module += ''.join(cell['source'][1:]) + '\n\n'
    # remove trailing spaces
    module = re.sub(r' +$', '', module, flags=re.MULTILINE)

    output_path = (fname.parent / 'exp')
    output_path.mkdir(parents=True, exist_ok=True)
    with open(output_path / fname_out, 'w') as f:
        f.write(module[:-2])

    print(f'Converted {fname} to {fname_out}')

if __name__ == '__main__': fire.Fire(notebook2script)
}
```

## Script run_notebook.py



```python
import nbformat, fire 
from nbconvert.preprocessors import ExecutePreprocessor

def run_notebook(path):
    """
    Run a Jupyter notebook and save the output.
    
    Args:
        path (str): Path to the notebook file.
    """
    with open(path) as f:
        nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)
    
    ExecutePreprocessor(timeout=600, kernel_name='python3').preprocess(nb, {})
    print('done')

if __name__ == '__main__':
    fire.Fire(run_notebook)

```

## Export

In [ ]:
#export
TEST = 'test'


In [ ]:
!python notebook2script.py 00_exports.ipynb


## Import

In [ ]:

from exp.nb_00 import TEST
print(f"value in imported TEST variable:  {TEST}")



# Testing functions

In [1]:
#export
import operator
def test(a, b, cmp, cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b), f"Test failed: {a} {cname} {b}"

def test_eq(a, b): test(a,b,operator.eq, '==') 

In [ ]:
test_eq(1, 1)
test_eq(1, 2)
test_eq(TEST, 'test')
test_eq(TEST, 'test1')


#